In [8]:
import os
import re
from configparser import ConfigParser, NoOptionError

import numpy as np
import pandas as pd
from bioblend.galaxy import GalaxyInstance
from bioblend.galaxy.tools import ToolClient
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import mapper, sessionmaker

In [2]:
galaxy_url = 'http://127.0.0.1:8080'
api_key = os.getenv('GALAXY_API_KEY')
gi = GalaxyInstance(url=galaxy_url, key=api_key)

### Connect to the database

In [3]:
galaxy_path = os.getenv('GALAXY_DIRECTORY')
config_file = os.path.join(galaxy_path, 'config/galaxy.ini.sample')
db_path = ''

if os.path.isfile(config_file):
    config = ConfigParser()
    config.read_file(open(config_file))
    
    try:
        # Get database path     
        db_path = config.get('app:main', 'database_connection')
#         db_path = 'postgresql://galaxy@localhost:5432/galaxy'  # temporary
            
    except NoOptionError:
        # Default
        db_path = 'sqlite:///' + os.path.join(galaxy_path, 'database/universe.sqlite')

engine = create_engine(db_path)

## Connections

In [4]:
query_get_connections = '''
SELECT DISTINCT ws.id, ws.workflow_id, ws.tool_id,
    ws.tool_version, ws.label, wsa.annotation,
    wsc.output_step_id AS in_port, wsc.input_step_id AS out_port

FROM workflow_step as ws, stored_workflow as sw

LEFT JOIN workflow_step_connection AS wsc
    ON ws.id = wsc.output_step_id

LEFT JOIN workflow_step_annotation_association as wsa
    ON ws.id = wsa.workflow_step_id

WHERE ws.workflow_id = sw.latest_workflow_id AND ws.type != 'data_input'
'''
connections = pd.read_sql(query_get_connections, con=engine)
connections[['in_port', 'out_port']] = connections[['in_port', 'out_port']].fillna(0.0).applymap(np.int64)

# def get_tool_ids(port):
#     for port_id in connections[port]:
#         tool_id = connections.tool_id[connections.id == port_id]
#         yield tool_id.values[0] if not tool_id.empty else None

# in_port_ids = pd.Series(list(get_tool_ids('in_port')), name='in_port_tool')
# out_port_ids = pd.Series(list(get_tool_ids('out_port')), name='out_port_tool')
# connections = pd.concat([connections, in_port_ids, out_port_ids], axis=1)
# connections = connections[(connections.in_port != 0) & (connections.out_port != 0)]

connections

,id,workflow_id,tool_id,tool_version,label,annotation,in_port,out_port
0,18,6,Cut1,1.0.2,cut 1,cut 1 annot,18,20
1,19,6,Cut1,1.0.2,cut 2,cut 2 annot,19,20
2,20,6,cat1,1.0.0,output,output annot,0,0


## Tool information

In [59]:
query_get_tools = '''
SELECT DISTINCT ws.tool_id AS id, ws.tool_version AS version
FROM workflow_step as ws, stored_workflow as sw
WHERE ws.tool_id IS NOT NULL AND ws.workflow_id = sw.latest_workflow_id
'''
workflow_tools = pd.read_sql(query_get_tools, con=engine)
workflow_tools['flag'] = 1

tc = ToolClient(gi)
all_tools = pd.DataFrame.from_dict(tc.get_tools())

tools = pd.merge(all_tools, workflow_tools, on=['id', 'version'], how='left')
tools = tools[pd.notnull(tools['flag'])].drop(['flag'], axis=1)

tools

,description,edam_operations,edam_topics,form_style,id,labels,link,min_width,model_class,name,panel_section_id,panel_section_name,target,tool_shed_repository,version
98,columns from a table,[],[],regular,Cut1,[],/tool_runner?tool_id=Cut1,-1,Tool,Cut,textutil,Text Manipulation,galaxy_main,NaN,1.0.2
118,tail-to-head,[],[],regular,cat1,[],/tool_runner?tool_id=cat1,-1,Tool,Concatenate datasets,textutil,Text Manipulation,galaxy_main,NaN,1.0.0


# ORM

In [65]:
class WorkflowStep:
    pass

class StoredWorkflow:
    pass

class WorkflowStepConnection:
    pass

class WorkflowStepAnnotationAssociation:
    pass

# Create SQLAlchemy Tables
metadata = MetaData(engine)
ws = Table('workflow_step', metadata, autoload=True)
sw = Table('stored_workflow', metadata, autoload=True)
wsc = Table('workflow_step_connection', metadata, autoload=True)
wsaa = Table('workflow_step_annotation_association', metadata, autoload=True)

mapper(WorkflowStep, ws)
mapper(StoredWorkflow, sw)
mapper(WorkflowStepConnection, wsc)
mapper(WorkflowStepAnnotationAssociation, wsaa)

# Create Session object
Session = sessionmaker(bind=engine)
session = Session()

In [81]:
connections_query = session.query(WorkflowStep.id, WorkflowStep.workflow_id,
                                  WorkflowStep.tool_id, WorkflowStep.tool_version,
                                  WorkflowStep.label,
                                  WorkflowStepAnnotationAssociation.annotation,
                                  WorkflowStepConnection.output_step_id.label('in_port'),
                                  WorkflowStepConnection.input_step_id.label('out_port')) \
                           .join(StoredWorkflow, WorkflowStep.workflow_id==\
                                StoredWorkflow.latest_workflow_id) \
                           .outerjoin(WorkflowStepConnection, WorkflowStep.id==\
                                WorkflowStepConnection.output_step_id) \
                           .outerjoin(WorkflowStepAnnotationAssociation, WorkflowStep.id==\
                                WorkflowStepAnnotationAssociation.workflow_step_id) \
                           .distinct().filter(WorkflowStep.type != 'data_input')

connections = pd.read_sql(connections_query.statement, session.bind)
connections[['in_port', 'out_port']] = connections[['in_port', 'out_port']].fillna(0.0).applymap(np.int64)

connections

,id,workflow_id,tool_id,tool_version,label,annotation,in_port,out_port
0,18,6,Cut1,1.0.2,cut 1,cut 1 annot,18,20
1,19,6,Cut1,1.0.2,cut 2,cut 2 annot,19,20
2,20,6,cat1,1.0.0,output,output annot,0,0


In [62]:
tools_query = session.query(WorkflowStep.tool_id.label('id'),
                         WorkflowStep.tool_version.label('version')) \
                     .join(StoredWorkflow, WorkflowStep.workflow_id==\
                               StoredWorkflow.latest_workflow_id) \
                     .distinct().filter(WorkflowStep.tool_id.isnot(None))

workflow_tools = pd.read_sql(tools_query.statement, session.bind)
workflow_tools['flag'] = 1

tc = ToolClient(gi)
all_tools = pd.DataFrame.from_dict(tc.get_tools())

tools = pd.merge(all_tools, workflow_tools, on=['id', 'version'], how='left')
tools = tools[pd.notnull(tools['flag'])].drop(['flag'], axis=1)

tools

,description,edam_operations,edam_topics,form_style,id,labels,link,min_width,model_class,name,panel_section_id,panel_section_name,target,tool_shed_repository,version
98,columns from a table,[],[],regular,Cut1,[],/tool_runner?tool_id=Cut1,-1,Tool,Cut,textutil,Text Manipulation,galaxy_main,NaN,1.0.2
118,tail-to-head,[],[],regular,cat1,[],/tool_runner?tool_id=cat1,-1,Tool,Concatenate datasets,textutil,Text Manipulation,galaxy_main,NaN,1.0.0
